In [1]:
import json
import pandas as pd
from datetime import datetime

In [7]:
df.head()

,UCsDmESjqNPukDmVnuneLrqw,UCY6Ij8zOds0WJEeqCLOnqOQ,UClN24S5-fc7LD0JJW0FdGWw,UC3iNdSV_RQU7DHTGpqEW96w,UC_52i6mlrzOZs_NEuSepzAg,UCr3cBLTYmIK9kY0F_OdFWFQ,UCV6g95OBbVtFmN9uiJzkFqQ,UCgCKYs56-LKEPGQ99DzqQOg,UCOwxx9VnEnlFKt5EB70KTzQ,UC2Ds30pkifFVD0CE08wF50g,...,UCte3qx7A5OWTPSnNjV4oheg,UCRei8TBpt4r0WPZ7MkiKmVg,UC6bXfhbiX1PsSi_-pdQeHjg,UC_7lgWbCA_HZQkaxmmQgNoA,UCQO_C1sfUCpUzzV-gxQ-lxA,UCHw-JjzU80GKrGJwI1Uf1OQ,UCDEaY2XwwfKomQAJgGfehWw,UCI-92EMS3ZJ11gz-S_oyaSA,UC7Mk9RmlKT_x8FAsMOJn1cg,UC9i7x4EzTBlNvslu1wWEB8Q
0,"[{'kind': 'youtube#searchListResponse', 'etag'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"[{'kind': 'youtube#searchListResponse', 'etag'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,"[{'kind': 'youtube#searchListResponse', 'etag'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,"[{'kind': 'youtube#searchListResponse', 'etag'...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,"[{'kind': 'youtube#searchListResponse', 'etag'...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
LIST_OF_CHANNEL_IDS = [
"UCsDmESjqNPukDmVnuneLrqw",
"UCY6Ij8zOds0WJEeqCLOnqOQ",
"UClN24S5-fc7LD0JJW0FdGWw",
"UC3iNdSV_RQU7DHTGpqEW96w",
"UC_52i6mlrzOZs_NEuSepzAg",
"UCr3cBLTYmIK9kY0F_OdFWFQ",
"UCV6g95OBbVtFmN9uiJzkFqQ",
"UCgCKYs56-LKEPGQ99DzqQOg",
"UCOwxx9VnEnlFKt5EB70KTzQ",
"UC2Ds30pkifFVD0CE08wF50g",
"UC7UGbBVrqLlq6CRxWWUmyKw",
"UCn1XB-jvmd9fXMzhiA6IR0w",
"UCFeqAfEuKm7lIg2ddQzh61A",
"UCAlxwHEm1bWWYoe5VCSQYEg",
"UCz7iJPVTBGX6DNO1RNI2Fcg",
"UCOAUz4d1NUNxSHh_SOJtdGQ",
"UCGGTAB19HlHEWPwwmxHsEKA",
"UCOsATJw-IZgqGT8MFrHjKGg",
"UCC-slLJZ4p4HOznMUcFn_2g",
"UCsKVP_4zQ877TEiH_Ih5yDQ",
"UCOinp9hALPUcZeoL3Kmaviw",
"UCGwu0nbY2wSkW8N-cghnLpA",
"UCu6v4AdYxVH5fhfq9mi5llA",
"UCj74rJ9Lgl3WTngq675wxKg",
"UCNIuvl7V8zACPpTmmNIqP2A",
"UCo8bcnLyZH8tBIH9V1mLgqQ",
"UCR0O-1cvuPNxDosvSDLpWHg",
"UCvUmwreRrbxeR1mbmojj8fg",
"UCUKi4zY5ETSqrKAjTBgjM-g",
"UCDQBZcjYKP1J1Nu-Y0_D37Q",
"UCQ9HvHH-KRYHI5ynj2kbLwQ",
"UCRfg0SWjIHm_h95e4V8X5og",
"UCWPB0WpnMIy-g7zncwIhvQg",
"UCj6CFdE3LSddaoszxq604TQ",
"UCXrtqz4fqf-OUEVpp2A1W0w",
"UChUg_Sd5i74SFoZH2XvagwQ",
"UC2hm5rD_IrfYRMfq5YQudgA",
"UC_nEHeUEVNY5ZYLRWg8KoZQ",
"UCrlzgjrKUa3AlrfSZTBG2fg",
"UCupWnlhB3PMW1w0CLfLCfug",
"UCsW0LA-ThH18OCrT9pa2zfQ",
"UCG-O5k7KKNPABULS20_MAMA",
"UCepPGz8AVCbggMl3BvboaBA",
"UCP21EOiKAkCGjv6kqS6UsZw",
"UC9z7EZAbkphEMg0SP7rw44A",
"UCZft8Ol-dMtguwam7bWItnw",
"UCYaxncRvOBu-vtdhxf6qciA",
"UCte3qx7A5OWTPSnNjV4oheg",
"UCRei8TBpt4r0WPZ7MkiKmVg",
"UC6bXfhbiX1PsSi_-pdQeHjg",
"UC_7lgWbCA_HZQkaxmmQgNoA",
"UCQO_C1sfUCpUzzV-gxQ-lxA",
"UCHw-JjzU80GKrGJwI1Uf1OQ",
"UCDEaY2XwwfKomQAJgGfehWw",
"UCI-92EMS3ZJ11gz-S_oyaSA",
"UC7Mk9RmlKT_x8FAsMOJn1cg",
"UC9i7x4EzTBlNvslu1wWEB8Q",
]

So I get the list of videos that a channel has. I don't yet have the specifics (aka metadata) on the videos yet because this API does not give me the information. What I have to do is filter this information to get a list of videos I need to throw at a different API to get the metadata. 

In [34]:
df = pd.read_json (r'searchListResponse.json')
channelIdToListOfVids = {} #channelId: []vids

#initialize the dicts
for channelId in LIST_OF_CHANNEL_IDS:
    channelIdToListOfVids[channelId] = []

# #youtuber
# #0~9ish: items
# #items: 0~46ish
count= 0

for index, midList in df.iterrows():  
    listOfLists = df.iat[index, index]
    for indList in listOfLists:
        for item in indList['items']:
            channelId = item['snippet']['channelId']
            if 'videoId' in item['id'].keys():
                snippet=item['snippet']
                vidId = item['id']['videoId']
                publishTime = snippet['publishTime']
                vidTitle = snippet['title']
                vidDesc = snippet['description']

                vidInfo = {'vidId' : vidId,
                'publishTime': publishTime,
                'vidTitle': vidTitle}
                channelIdToListOfVids[channelId].append(vidInfo)
                count+=1

print(len(channelIdToListOfVids.keys()), " keys detected")
vidCont=0
for i in channelIdToListOfVids.keys():
    vidCont+=len(channelIdToListOfVids[i])
print(count, " videos detected")
# channelIdToListOfVids

57  keys detected
4973  videos detected


We now have the list of videoIds for each channel

After using this information in `main.py` to shoot at the API to get individual video metadata, I again filter out information specific to each video.

In [42]:
df = pd.read_json (r'videoListResponse.json')
backend_purposes= {}

def flattenList(list):
    flat_list = [item for sublist in list for item in sublist]
    return flat_list
    
for channelId in LIST_OF_CHANNEL_IDS:
    backend_purposes[channelId]=[]

matching_vidId_to_info={}
count = 0
debugMsg = []
setOfTypes = []
for index, indList in df.iterrows():
    items = indList['items']
    for item in items:
        count+=1
        vidId = item['id']
        channelId = item['snippet']['channelId']
        contentDetails= item['contentDetails']
        duration= contentDetails['duration']
        statistics = item['statistics']
        viewCount = statistics['viewCount']
        likeCount = statistics.get('likeCount')
        commentCount = statistics.get('commentCount')
        description = item['snippet']['description']
        if (likeCount == None or commentCount == None):
            print(vidId, "has", "likes" if likeCount == None else '', "comments" if commentCount == None else '', "disabled")
        topicCategories = []
        if item.get('topicDetails') != None:
            topicCategories = item.get('topicDetails').get('topicCategories')
            setOfTypes.append([i for i in topicCategories])
        if True not in ['music' in strng.lower() for strng in topicCategories] or item.get('topicDetails') == None: #all videos that are not music-typed or just don't have a type
            matching_vidId_to_info[vidId] = {'duration': duration,
                            'viewCount': int(-1 if viewCount is None else viewCount),
                            'likeCount': int(-1 if likeCount is None else likeCount),
                            'commentCount': int(-1 if commentCount is None else commentCount),
                            'topicCategories': topicCategories,
                            'description': description}
            backend_purposes[channelId].append(vidId)
        else: #all videos that are music-typed
            debugMsg.append(vidId)

vidIdsCollected =0 
for i in backend_purposes.keys():
    vidIdsCollected += len(backend_purposes[i])
print(vidIdsCollected, " videos to run semantic analysis on")
print(set(flattenList(setOfTypes)))
print("There were ", 4971-len(matching_vidId_to_info), " videos categorized as music-videos")


QQilWUln3F8 has  comments disabled
0eCXTYobXQk has  comments disabled
hpQQohcHk9Q has  comments disabled
fV15ba-dpPI has  comments disabled
Cb6F14AGrvI has  comments disabled
OmpK2tkKmNQ has likes  disabled
SuApY0sEvz4 has  comments disabled
XJcM__haurU has  comments disabled
BMf9QEg3F-I has  comments disabled
LcyCs7fRmgc has  comments disabled
J10KDPg_Im0 has  comments disabled
7ghz_npkbmE has  comments disabled
JkGZ2Hl1l8c has  comments disabled
7CjAIbmZZGY has  comments disabled
bNMtxQnnuaA has  comments disabled
RlU32AfEVeU has likes  disabled
4774  videos to run semantic analysis on
{'https://en.wikipedia.org/wiki/Tennis', 'https://en.wikipedia.org/wiki/Music', 'https://en.wikipedia.org/wiki/Christian_music', 'https://en.wikipedia.org/wiki/Video_game_culture', 'https://en.wikipedia.org/wiki/Jazz', 'https://en.wikipedia.org/wiki/Health', 'https://en.wikipedia.org/wiki/Rhythm_and_blues', 'https://en.wikipedia.org/wiki/Film', 'https://en.wikipedia.org/wiki/Hobby', 'https://en.wikiped

In [43]:
#deepcopy
channelIdToListOfVids_copy = json.loads(json.dumps(channelIdToListOfVids))
debugMsg = []
for channelId in LIST_OF_CHANNEL_IDS:
    listOfVideosForEachChannel = channelIdToListOfVids_copy[channelId]
    for video in listOfVideosForEachChannel:
        additionalVideoMetadata= matching_vidId_to_info.get(video['vidId'])
        if additionalVideoMetadata:
            video.update(additionalVideoMetadata)
        else:
            debugMsg.append(video['vidId'])

print(sum(len(channelIdToListOfVids_copy[channelId]) for channelId in LIST_OF_CHANNEL_IDS))
print(len(debugMsg))
#some of these vids are private, duplicates. 

4973
199


In [44]:
listOfVideoIds=[]
for key, value in backend_purposes.items():
    listOfVideoIds+=value
with open('listOfVideoIds.json', 'w') as outfile:
    json.dump(listOfVideoIds, outfile)

In [45]:
REFERENCE_UTC_TIME_FOR_PUBLISH = datetime(2021, 12, 24, 5, 2, 59, 166993)
channelId_to_earliestVideoTimestamp_and_mostViewedVideo = {} # {channelId: {firstVideoTimestamp: <>, mostViewedVideoInfo: {}}}

#channelId_to_representative_vid_and_first_vid
for channelId in LIST_OF_CHANNEL_IDS:
    listOfVideoInfo = channelIdToListOfVids_copy[channelId]
    df = pd.DataFrame(list(listOfVideoInfo))
    mostViewedVideoIndex = df['viewCount'].idxmax()
    earliestVideoTimestamp = min(df['publishTime'])
    firstUploadDate = earliestVideoTimestamp
    daysLiveSinceFirstUpload = pd.Timedelta(REFERENCE_UTC_TIME_FOR_PUBLISH - datetime.strptime(earliestVideoTimestamp, '%Y-%m-%d'+'T'+'%H:%M:%S'+'Z')).round('d')
    channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId] = {'firstUploadDate': firstUploadDate, 'daysSinceFirstVideo': daysLiveSinceFirstUpload, 'mostViewedVideo': df.iloc[mostViewedVideoIndex].to_dict()}

And now, I update the dictionary of each video specified by vidId, to update the video information with more metadata. Not only does the video info contain `vid publishedAt`, `publishTime`, `vidTitle`, and `vidDescription`, but it also contains `duration`, `viewCount`, `likeCount`, `commentCount`, and `topicCategories`

In [46]:
with open('masterSheet.json', 'w') as outfile:
    json.dump(channelIdToListOfVids_copy, outfile)

channelIdToListOfVidIds = {}
for channelId in LIST_OF_CHANNEL_IDS:
    vidInfoList = channelIdToListOfVids_copy[channelId]
    channelIdToListOfVidIds[channelId] = []
    for vidInfo in vidInfoList:
        channelIdToListOfVidIds[channelId].append(vidInfo['vidId'])

with open('masterSheetOfChannelIdToListOfVidIds.json', 'w') as outfile:
    json.dump(channelIdToListOfVidIds, outfile)

If I had a lot of resources like an actual GPU and memory, I'd probably analyze all the videos to find the average video to be representative of the channel, but since I don't, I want to take the most viewed video of each channel. This is a decision I made based on the... untestable hypothesis that the most-viewed video is the one that brought the channel the most subscriber count out of all of them. 

So I think taking the most-viewed video to represent the channel is an acceptable alternative. Thus, I will begin searching for 
1. the most viewed video by searching for `max(viewcount)` for each Channel
2. the first uploaded video by searching for `min(publishedAt)` for each channel because I need those stats as well. 

The result I get should be something along the lines of: `{ChannelId: 'xyz', earliestVideoPublishedDate: 'DateTime', mostViewedVideo : {vidInfo dictionary}}`

In [47]:
listOfVideoInfo = channelIdToListOfVids_copy["UC9z7EZAbkphEMg0SP7rw44A"]
df = pd.DataFrame(list(listOfVideoInfo)) 
df

,vidId,publishTime,vidTitle,duration,viewCount,likeCount,commentCount,topicCategories,description
0,42OleX0HR4E,2019-01-18T00:16:23Z,Cary teaches you how to time travel,PT2M45S,3393376.0,158411.0,20719.0,[https://en.wikipedia.org/wiki/Knowledge],"hello hello, in this educational video I will ..."
1,DxQQwExTjcM,2014-05-04T22:31:04Z,How strong is Steve? (Reuploaded again),PT2M12S,5302780.0,141944.0,29888.0,[https://en.wikipedia.org/wiki/Video_game_cult...,Original version (this video was first made in...
2,31dsH2Fs1IQ,2015-06-25T18:35:48Z,Evolution Simulator (Part 2/4),PT4M46S,2285774.0,28634.0,717.0,[https://en.wikipedia.org/wiki/Knowledge],Part 1: https://www.youtube.com/watch?v=GOFws_...
3,uaGEjrADGPA,2012-02-02T02:27:24Z,The Scale of the Universe 2,PT3M24S,2238073.0,17968.0,2144.0,[https://en.wikipedia.org/wiki/Knowledge],**Interactives**\nMobile Version:\nhttps://itu...
4,FlfOcIDRKPQ,2015-08-04T02:43:51Z,No More Lasers!,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
266,SKQJlvqhDzQ,2016-03-26T21:29:04Z,"GOLAD 3/26: undo/redo, symmetry",PT8M20S,8637.0,161.0,104.0,[https://en.wikipedia.org/wiki/Knowledge],VSauce — Michael here. Another dev vlog!\nⒼⓄⓁⒶ...
267,7SXr4u2L1yg,2011-12-17T22:18:23Z,My Minecraft Mod: Diamond Trees!,PT13M,7796.0,83.0,29.0,[https://en.wikipedia.org/wiki/Action-adventur...,The diamonds are finally harvested at 9:00\n\n...
268,6KrxVYo0mBQ,2012-03-05T05:27:14Z,How many comments?,PT17S,10344.0,77.0,759.0,[],This is an experiment. What will happen? Let...
269,mvzqt-BpS7w,2012-06-16T04:19:09Z,How do cats look out of windows?,PT4M19S,6112.0,97.0,41.0,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,Here's how.


So... creating a dataframe out of the info is quite trivial. Really easy. I also just noticed that some of these youtube channels are going to have music videos... which is not in the scope of my project right now. Gonna have to remove any videos with topicCategories of music themes.

In [48]:
REFERENCE_UTC_TIME_FOR_PUBLISH = datetime(2021, 12, 24, 5, 2, 59, 166993)
channelId_to_earliestVideoTimestamp_and_mostViewedVideo = {} # {channelId: {firstVideoTimestamp: <>, mostViewedVideoInfo: {}}}

#channelId_to_representative_vid_and_first_vid
for channelId in LIST_OF_CHANNEL_IDS:
    listOfVideoInfo = channelIdToListOfVids_copy[channelId]
    df = pd.DataFrame(list(listOfVideoInfo))
    mostViewedVideoIndex = df['viewCount'].idxmax()
    earliestVideoTimestamp = min(df['publishTime'])
    firstUploadDate = earliestVideoTimestamp
    daysLiveSinceFirstUpload = pd.Timedelta(REFERENCE_UTC_TIME_FOR_PUBLISH - datetime.strptime(earliestVideoTimestamp, '%Y-%m-%d'+'T'+'%H:%M:%S'+'Z')).round('d')
    channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId] = {'firstUploadDate': firstUploadDate, 'daysSinceFirstVideo': daysLiveSinceFirstUpload, 'mostViewedVideo': df.iloc[mostViewedVideoIndex].to_dict()}

In [49]:
def try_parsing_date(text):
    for fmt in ('%Y-%m-%d'+'T'+'%H:%M:%S'+'Z', '%Y-%m-%d'+'T'+'%H:%M.%S'+'Z'):
        try:
            return datetime.strptime(text, fmt)
        except ValueError:
            pass
            
    raise ValueError('no valid date format found')

In [50]:
#important. date ref is 12/11 because this is when I pulled this data. 

df = pd.read_json (r'channelListResponse.json')
REFERENCE_UTC_NOW= datetime(2021, 12, 24, 5, 2, 59, 166993)
channelInfo_master_df= pd.DataFrame()

for index, row in df.iterrows():
    items=df['items'][index][0]
    snippet = items['snippet']

    title=snippet['title']
    channelId=items['id']
    publishedAt=snippet['publishedAt']
    description=snippet['description']
    viewCount=items['statistics']['viewCount']
    subscriberCount=items['statistics']['subscriberCount']
    videoCount=items['statistics']['videoCount']
    daysLive = pd.Timedelta(REFERENCE_UTC_NOW - try_parsing_date(publishedAt)).round('d').days

    data = {'channelName':title, 
    'channelId': channelId, 
    'daysLiveAsOfLaunch': daysLive,
    'daysSinceFirstVideo': channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['daysSinceFirstVideo'].days,
    'description': description,
    'viewCount': int(viewCount),
    'subscriberCount': int(subscriberCount),
    'videoCount': int(videoCount),
    'firstUploadDate' : channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['firstUploadDate'],
    'subscribersPerYearSinceFirstUpload' : float(int(subscriberCount)/(channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['daysSinceFirstVideo'].days/365)),
    'viewCountPerYearSinceFirstUpload' : float(int(viewCount)/(channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['daysSinceFirstVideo'].days/365)),
    }
    df_generated = pd.DataFrame(data, index=[index])
    channelInfo_master_df= channelInfo_master_df.append(df_generated)
channelInfo_master_df.head()

,channelName,channelId,daysLiveAsOfLaunch,daysSinceFirstVideo,description,viewCount,subscriberCount,videoCount,firstUploadDate,subscribersPerYearSinceFirstUpload,viewCountPerYearSinceFirstUpload
0,Alex Clark,UCsDmESjqNPukDmVnuneLrqw,4404,4399,Funny vids about my life. I'm thankful for th...,471572066,4270000,318,2009-12-07T21:13:28Z,354296.431007,3.912794e+07
1,Alex Meyers,UCY6Ij8zOds0WJEeqCLOnqOQ,2360,2351,I make cartoons and sometimes video essays abo...,476584688,2880000,258,2015-07-18T11:43:37Z,447128.881327,7.399124e+07
2,WowRightMeow,UClN24S5-fc7LD0JJW0FdGWw,1958,1819,"Yo! I'm Jamie, a dorky human from Wales. Did a...",35623911,561000,33,2016-12-31T15:08:07Z,112570.093458,7.148283e+06
3,Andrei Terbea,UC3iNdSV_RQU7DHTGpqEW96w,3159,3159,Doing animated commentary and solving crimes w...,293487480,2860000,136,2013-04-30T20:01:32Z,330452.674897,3.391039e+07
4,Billy But Better,UC_52i6mlrzOZs_NEuSepzAg,2817,1746,"Yes, it's a foot.\n\nSocial Medias:\nTwitter -...",12167810,260000,23,2017-03-14T02:31:14Z,54352.806415,2.543672e+06


In [51]:
with open("channelHolisticInfo.json", 'w') as f:
    json.dump(channelInfo_master_df.to_dict(), f)

`MostViewedVideoInfo` is showing up as NaN because it's of type `dict`. So need to just make sure I have access to the deets.

In [52]:
mostViewedVideo_master_dict = {}
for channelId in LIST_OF_CHANNEL_IDS:
    data = channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['mostViewedVideo']
    mostViewedVideo_master_dict[channelId]= data
    
mostViewedVideo_master_df = pd.DataFrame.from_dict(mostViewedVideo_master_dict).T
mostViewedVideo_master_df

,vidId,publishTime,vidTitle,duration,viewCount,likeCount,commentCount,topicCategories,description
UCsDmESjqNPukDmVnuneLrqw,EcgkRp2IUsc,2017-10-26T01:05:22Z,My Hot Babysitter Got Me In Trouble,PT6M30S,35958011.0,840535.0,75600.0,"[https://en.wikipedia.org/wiki/Entertainment, ...",First Name Babysitter. Last Name Trouble. \nGE...
UCY6Ij8zOds0WJEeqCLOnqOQ,kbCah6yhYRs,2019-06-11T11:00:07Z,Descendants doesn&#39;t make any sense...,PT16M8S,9806120.0,262432.0,22460.0,"[https://en.wikipedia.org/wiki/Entertainment, ...",Descendants doesn't make any sense animation\n...
UClN24S5-fc7LD0JJW0FdGWw,ewsGmhAjjjI,2018-10-09T20:50:50Z,I have APHANTASIA (and you may too...without r...,PT12M54S,6026056.0,319132.0,71950.0,"[https://en.wikipedia.org/wiki/Hobby, https://...",So...yeah. I'm an artist that can't see pictur...
UC3iNdSV_RQU7DHTGpqEW96w,gA0bi-bFEYs,2018-06-22T19:20:36Z,How KSI Saved Logan Paul&#39;s Career,PT6M48S,20317731.0,513706.0,95771.0,"[https://en.wikipedia.org/wiki/Entertainment, ...",This KSI vs Logan Paul fight is interesting to...
UC_52i6mlrzOZs_NEuSepzAg,so1_5hYUEE8,2017-09-20T22:12:54Z,Being a Boba Barista (Work Stories),PT5M20S,2196478,58179,3759,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,Get it? “B”eing a “B”oba “B”arista “b”y “B”ill...
UCr3cBLTYmIK9kY0F_OdFWFQ,xa-4IAR_9Yw,2016-03-29T09:20:12Z,Casually Explained: Is She Into You?,PT2M31S,17115808,736729,20174,"[https://en.wikipedia.org/wiki/Entertainment, ...",Based on a true story. Dedicated to Jimmy. Unr...
UCV6g95OBbVtFmN9uiJzkFqQ,I9uWUw1fxOY,2020-10-23T15:59:47Z,The Life of a Speedrunner,PT1M1S,19333474.0,833043.0,20375.0,[https://en.wikipedia.org/wiki/Video_game_cult...,"ANOTHER DAY, ANOTHER CHANCE TO OPTIMIZE THIS R..."
UCgCKYs56-LKEPGQ99DzqQOg,W8P5ewPk9fM,2019-01-18T20:00:06Z,Why I Dont Want a Hot Wife (Ft. Emirichu),PT8M7S,2318935,115419,5846,[https://en.wikipedia.org/wiki/Entertainment],"This week, we talk about one of the things I g..."
UCOwxx9VnEnlFKt5EB70KTzQ,12Ne9n40tmw,2017-02-25T19:46:48Z,WHO&#39;S YOUR DADDY? ANIMATED with Pewdiepie ...,PT2M26S,11704475,246749,6180,[],Animating Pewdiepie and Jacksepticeye's who's ...
UC2Ds30pkifFVD0CE08wF50g,Mv8OkBjySGQ,2017-09-15T00:08:53Z,My Super HOT English Teacher (Animation),PT8M5S,9249114,303683,26328,"[https://en.wikipedia.org/wiki/Entertainment, ...","I have so many more stories, so subscribe and ..."


So I need to remove... all the music-themed game letsplay stuff. And I did. All the way up there.

Whew. And now... I need to gather the transcripts for the 33 videos above to run semantic analysis on it. There's probably an API for it...

In [54]:
channelId_to_mostViewedVidId = {}

for channelId in LIST_OF_CHANNEL_IDS:
    mostViewedVidId = mostViewedVideo_master_dict[channelId]['vidId']
    channelId_to_mostViewedVidId[channelId] = mostViewedVidId

with open('channelId_to_mostViewedVidId.json', 'w') as outfile:
    json.dump(channelId_to_mostViewedVidId, outfile)

Ah. I am also interested in seeing when youtubers got their 100k milestone. Actually, the entire premise of this project was initially, me trying to find a link between their artstyle and historical sub count. 
Unfortunately, there isn't a way to get the historical subcount. Social blade does not manage subscriber count back in time, and I'm not sure if Youtube does, but even if it does, it's not available to the public. There's no APIs for the information. 

SO. I'm gonna go the cheap but mostly reliable route. Cheap because I'm going to be looking for `0k` in video descriptoins and titles, reliable because although self-reported, the youtuber has a growing fanbase.

In [32]:
count = 0

for channelId in LIST_OF_CHANNEL_IDS:
    listOfVidInfo = channelIdToListOfVids_copy[channelId]
    for vidInfo in listOfVidInfo:
        vidDescription = vidInfo['vidDescription']
        vidTitle = vidInfo['vidTitle']
        if vidDescription.find("0k") >= 0:
            count +=1
            print(vidDescription)
        if vidTitle.find("0k") >= 0 :
            count +=1
            print(vidTitle)
print(count)

To vote for me to make more cartoons click the vote button here(I'll win $250000k): http://bit.ly/grantplease Why are we totally ...
My Youtube Story  (30k Giveaway)
How I make my animation videos I really don't know what to say. I had completely given up on ever getting 50k, and 100k was just ...
How I make my videos (100k subscriber special)
MONSTER LEGENDS: https://monsterlegends.onelink.me/QSYm/AndreiTerbea Download before July 3rd and get 50k Food, 300k ...
This year has been crazy. We started with somewhere around 30k subscribers and we're finishing off the year with almost 1.3 ...
Yesterday we hit 200k subscribers and i made the super last minute decision to go live on Twitch and draw people in the chat as a ...
100k Subs + Anime North Recap
100k is a lot of nerds. A special thanks to Berd for Beh-ing in this video and also to Aaron at Nevercake for the nice poem and a ...
come see us draw like, everything Jazza's video: https://www.youtube.com/watch?v=0kI2JcvEDGw&ab_channel=Dra

and then I remembered this... figment of memory.
I think I saw a History section in wikitubia which tells you approximately when their subscriber milestone was.
alright. I guess we're doing this. This is actually going to be partially a webscraping project.
And this is quickly getting out of hand. I need to define the scope of this side project.

And... I should probably plan things out better. So... 
get channelId-> get their youtube videos by next token-> get video info. 

get transcripts
get mp4
get historical sub data.
get artstyle

Outcome. Let's limit it to 5. Use discretion when displaying data.
1. Historical subCount, normalized. Other easy stuff: average length of video (sum(Duration)/videoCount)
2. nlp (how positive/ negative is their content by transcripts)? word bubbles.
3. artstyle (colorized, less colorized, motion, shots)

============================================================================================================

***1. Dataframe data*** <br/>
By channelId:
* how long did it take them to get to 100k? What kind of videos did they publish before hitting 100k? I need chronological info.. 
--> 
    * average duration of video
    * subCount normalized
    * viewCount normalized

***2. NLP how positive/ negative is it?*** <br/>
By channelId:
* semantic analysis
* word bubble
* positivity/ negativity


***3. Artstyle*** <br/>
By channelId:
* color
* shots + shot length

***Conclusion***
1. Is there correlation between artstyle and sub count?
2. Is there a correlation between semantics + sub count?
3. Is there a common pattern that has gotten these animation youtubers to where they are today?